### OICM Inference for models deployed with: vLLM and TGI
##### ref: https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html

In [53]:
import requests
import json
import os
from openai import OpenAI

In [11]:
api_key = os.getenv('OI_API_KEY') # Your valid api key
model_version_id = os.getenv('OI_MODEL_VERSION_ID') # model version id
base_url =f"" # your valid model url
# example: https://inference.{oicm_host}/models/{model_version_id}/proxy/vllm/v1
# oicm_host: is the platform url ex: develop.openinnovation.ai

model_name = "tiiuae/Falcon3-1B-Instruct" # HF Model name


api_key = "sk-jpZk1U1b5C5zmlpYRtA_cw9RIAzhQQccvU7hu_d6ZnU"
model_version_id = "42c4f31d-9f7d-4f64-b8ad-3c63ada3df53"
base_url = f"https://inference.develop.openinnovation.ai/models/{model_version_id}/proxy/v1"

headers = {
    "Authorization": f"Bearer {api_key}"
}

### Using OpenAI API Client

In [14]:
client = OpenAI(
    base_url=base_url,
    api_key=api_key
)

In [16]:
stream = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You're a helpful assistant"},
                {"role": "user", "content": "what do you know about Egypt?"}
                
            ],
            max_tokens=256,
            temperature=0.7,
            stream=True
        )
        
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

Egypt is a country located in Northeast Africa, known for its rich history and cultural heritage. It is one of the oldest civilizations in the world, having developed along the fertile banks of the Nile River. Ancient Egyptian civilization flourished for thousands of years, leaving behind iconic structures such as the Pyramids of Giza and the Sphinx.

Egypt is also renowned for its contributions to art, literature, and science. The Rosetta Stone, discovered in 1799, played a crucial role in the deciphering of Egyptian hieroglyphics, which has provided invaluable insights into ancient Egyptian culture and history.

The country also boasts a vibrant modern culture, with a diverse food scene, including iconic dishes like Egyptian kebabs and falafel. Egypt is also home to numerous museums, including the Museum of Egyptian Civilization in Cairo, which showcases the archaeological treasures from various periods of Egyptian history.

Additionally, Egypt is a popular tourist destination known 

### Using requests

In [19]:
payload = {
    "messages": [
        {"role": "system", "content": "You're a helpful assistant"},
        {"role": "user", "content": "what do you know about Egypt?"}
    ],
    "max_tokens": 256,
    "temperature": 0.7,
    "model": model_name,
    "stream": True
}

response = requests.post(f"{base_url}/chat/completions", json=payload, headers=headers, stream=True)

for token in response.iter_lines():
    try:
        decoded_token = token.decode("UTF-8")
        json_str = decoded_token[6:]
        token_json = json.loads(json_str)
        new_str = token_json['choices'][0]['delta']['content']
        print(new_str, end="", flush=True)
    except:
        pass

Egypt is a country located in the northeastern corner of Africa, bordered by Libya to the north, Sudan to the east, South Sudan to the south, and Israel and Palestine to the southwest. It is one of the oldest civilizations in the world, with a rich history dating back over 5000 years, and it is famous for its iconic landmarks like the Pyramids of Giza, the Sphinx, and the Nile River.

Egypt is located on the east bank of the Nile River, which is the longest river in Africa. The Nile has played a crucial role in the history and development of Egypt, providing fertile soil for agriculture and sustaining the Egyptian civilization.

The country is divided into two main regions: the Nile Valley, which includes the fertile agricultural lands along the river, and the Western Desert, which is arid and rocky.

Egypt is known for its diverse culture and heritage, with a mix of ancient Egyptian traditions and influences from other civilizations over the centuries. It has a robust economy, driven 

### Integrate with Langchain

In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
        model_name=model_name,
        openai_api_key=api_key,
        openai_api_base=base_url,
        max_tokens=1000,
)

llm_answer = llm.invoke("Egypt is")

llm_answer.content

"Egypt is a country located in Northeast Africa, bordered by Sudan to the north, South Sudan to the east, Ethiopia to the northwest, and Libya to the west. It is also connected to the Mediterranean Sea to the south through the Suez Canal. The capital of Egypt is Cairo, which is also the largest city in the country. The modern nation-state of Egypt was formed in 1922 following the Arab nationalist struggle for independence from British colonial rule. Egypt has a rich history spanning over 5,000 years, making it one of the world's oldest civilizations."

### Langchain Structured Output
#### Structured output is only supported with models deployed with vLLM

In [44]:
from typing import List, Optional
from pydantic import BaseModel, Field

In [46]:
class BookRecommendation(BaseModel):
    """Details of a book recommendation."""
    title: str = Field(description="The title of the book")
    author: str = Field(description="The author of the book")
    genre: str = Field(description="The genre of the book")
    rating: float = Field(description="The book's rating on a scale from 1.0 to 5.0")
    reasons: List[str] = Field(
        description="Reasons why this book is recommended"
    )

In [50]:
structured_llm = llm.with_structured_output(BookRecommendation)

response = structured_llm.invoke("Recommend a good science fiction book for someone who loves space exploration.")

response

BookRecommendation(title='The Expanse', author='James S. A. Corey', genre='Science Fiction', rating=8.0, reasons=['The Expanse is a critically acclaimed series that explores a future where humanity has colonized the solar system and faces challenges such as resource scarcity, political conflict, and alien threats.', 'It delves into the complexities of human nature, ethics, and survival in a universe where boundaries blur.', 'Its rich world-building and intricate plotlines make it a compelling read for fans of space exploration and speculative fiction.', 'The series offers a deep dive into the consequences of human actions in a universe governed by gravity, technology, and the stars.', 'Readers will find the blend of suspense, character development, and political intrigue engaging and thought-provoking.', "Overall, 'The Expanse' is a solid choice for those who enjoy the depth and complexity of science fiction narratives set in the vast expanse of the cosmos."])